In [11]:
import pandas as pd

In [12]:
data_df = pd.read_csv("../data/original_data.csv")

In [13]:
data_df.duplicated().sum()

0

In [14]:
data_df = data_df[data_df['Description'].notna()].reset_index(drop=True)

In [15]:
# Split Category strings into lists (split on comma, strip whitespace)
def parse_categories(val):
    if pd.isna(val):
        return []
    if isinstance(val, str):
        parts = [p.strip() for p in val.split(',') if p.strip()]
        return parts
    if isinstance(val, list):
        return val
    return [str(val)]

data_df['Category'] = data_df['Category'].apply(parse_categories)

In [16]:
data_df

,Title,Authors,Description,Category,Publisher,Price Starting With ($),Publish Date (Month),Publish Date (Year)
0,Journey Through Heartsongs,"By Stepanek, Mattie J. T.",Collects poems written by the eleven-year-old ...,"[Poetry, General]",VSP Books,19.96,September,2001
1,In Search of Melancholy Baby,"By Aksyonov, Vassily, Heim, Michael Henry, and...",The Russian author offers an affectionate chro...,"[Biography & Autobiography, General]",Random House,4.99,June,1987
2,The Dieter's Guide to Weight Loss During Sex,"By Smith, Richard","A humor classic, this tongue-in-cheek diet pla...","[Health & Fitness, Diet & Nutrition, Diets]",Workman Publishing Company,4.99,January,1978
3,Germs : Biological Weapons and America's Secre...,"By Miller, Judith, Engelberg, Stephen, and Bro...","Deadly germs sprayed in shopping malls, bomb-l...","[Technology & Engineering, Military Science]",Simon & Schuster,4.99,October,2001
4,The Good Book: Reading the Bible with Mind and...,"By Gomes, Peter J.","""The Bible and the social and moral consequenc...","[Religion, Biblical Biography, General]",Harper Perennial,5.29,May,1998
...,...,...,...,...,...,...,...,...
70199,Like A Sister: A Novel,"By Daugharty, Janice",Sister cannot say exactly when or where she wa...,"[Fiction, Literary]",Harper,5.37,November,1999
70200,Creating Web Pages Simplified (3-D Visual Series),"By Maran, Ruth, Whitehead, Paul, and Marangrap...","An ""owner's manual"" for first-time Web page cr...","[Computers, Internet, General]",Hungry Minds Inc,5.95,January,1997
70201,EVA: The Real Key to Creating Wealth,"By Ehrbar, Al","Called ""today's hottest financial idea and get...","[Business & Economics, Corporate Finance, Gene...",Wiley,29.96,October,1998
70202,The Essentials of Spanish (REA's Language Seri...,"By Mouat, Ricardo Gutierrez",REA’s Essentials provide quick and easy access...,"[Foreign Language Study, Spanish]",Research & Education Association,5.29,January,1998


In [ ]:
data_df['Authors'] = data_df['Authors'].apply(lambda x: x.replace("By ", "") if pd.notna(x) else x)

In [42]:
# Filter out rows with invalid author formats (all uppercase, no commas)
def is_valid_author_format(val):
    if pd.isna(val):
        return True  # Keep NaN values for now
    
    val_str = str(val).strip()
    
    # Check if string is all uppercase (likely invalid format)
    if val_str.isupper():
        return False
    
    # Must contain at least one comma for proper "Lastname, Firstname" format
    if ',' not in val_str:
        return False
    
    return True

data_df = data_df[data_df['Authors'].apply(is_valid_author_format)].reset_index(drop=True)

In [54]:
import re

def parse_authors(val):
    if pd.isna(val):
        return []
    
    # First split by " and " to separate multiple authors
    authors = re.split(r'\s+and\s+', str(val))
    
    # Then split each author segment by comma
    all_authors = []
    for author in authors:
        parts = author.split(',')
        parts = [name.strip() for name in parts if name.strip()]
        all_authors.extend(parts)
    
    # Filter out parts that are ONLY known titles/roles or contain museum/institution keywords
    # Common academic/editorial roles: EDT, COR, ILT, TRN, FRW, etc.
    filtered_authors = []
    known_titles = ['EDT', 'COR', 'ILT', 'TRN', 'FRW', 'PHD', 'DR', 'MD', 'PROF', 'ESQ']
    institution_keywords = ['MUSEUM', 'LIBRARY', 'SOCIETY', 'INSTITUTE', 'FOUNDATION', 'ASSOCIATION', 'UNIVERSITY']
    
    for author in all_authors:
        stripped = author.strip()
        # Remove parentheses and dots for comparison
        cleaned = re.sub(r'[\(\)\.]', '', stripped).upper().strip()
        
        # Skip only if it exactly matches a known title
        if cleaned in known_titles:
            continue
        
        # Skip if it contains institution keywords
        if any(keyword in stripped.upper() for keyword in institution_keywords):
            continue
        
        # Remove parentheses and their contents from the author name
        cleaned_author = re.sub(r'\s*\([^)]*\)', '', author).strip()
        if cleaned_author:  # Only add if there's something left after removing parentheses
            filtered_authors.append(cleaned_author)
    
    # Join every two consecutive elements to form full names (Lastname + Firstname)
    full_names = []
    for i in range(0, len(filtered_authors), 2):
        if i + 1 < len(filtered_authors):
            # Join lastname and firstname
            full_names.append(f"{filtered_authors[i]}, {filtered_authors[i+1]}")
        else:
            # Odd number of parts, keep the last one as is
            full_names.append(filtered_authors[i])
    
    return full_names

In [57]:
data_df['Authors'] = data_df['Authors'].apply(parse_authors)

In [ ]:
data_df.to_csv("../data/processed_data.csv", index=False)

,Title,Authors,Description,Category,Publisher,Price Starting With ($),Publish Date (Month),Publish Date (Year)
0,Journey Through Heartsongs,"[Stepanek, Mattie J. T.]",Collects poems written by the eleven-year-old ...,"[Poetry, General]",VSP Books,19.96,September,2001
1,In Search of Melancholy Baby,"[Aksyonov, Vassily, Heim, Michael Henry, Bouis...",The Russian author offers an affectionate chro...,"[Biography & Autobiography, General]",Random House,4.99,June,1987
2,The Dieter's Guide to Weight Loss During Sex,"[Smith, Richard]","A humor classic, this tongue-in-cheek diet pla...","[Health & Fitness, Diet & Nutrition, Diets]",Workman Publishing Company,4.99,January,1978
3,Germs : Biological Weapons and America's Secre...,"[Miller, Judith, Engelberg, Stephen, Broad, Wi...","Deadly germs sprayed in shopping malls, bomb-l...","[Technology & Engineering, Military Science]",Simon & Schuster,4.99,October,2001
4,The Good Book: Reading the Bible with Mind and...,"[Gomes, Peter J.]","""The Bible and the social and moral consequenc...","[Religion, Biblical Biography, General]",Harper Perennial,5.29,May,1998
...,...,...,...,...,...,...,...,...
67650,Like A Sister: A Novel,"[Daugharty, Janice]",Sister cannot say exactly when or where she wa...,"[Fiction, Literary]",Harper,5.37,November,1999
67651,Creating Web Pages Simplified (3-D Visual Series),"[Maran, Ruth, Whitehead, Paul, Marangraphics I...","An ""owner's manual"" for first-time Web page cr...","[Computers, Internet, General]",Hungry Minds Inc,5.95,January,1997
67652,EVA: The Real Key to Creating Wealth,"[Ehrbar, Al]","Called ""today's hottest financial idea and get...","[Business & Economics, Corporate Finance, Gene...",Wiley,29.96,October,1998
67653,The Essentials of Spanish (REA's Language Seri...,"[Mouat, Ricardo Gutierrez]",REA’s Essentials provide quick and easy access...,"[Foreign Language Study, Spanish]",Research & Education Association,5.29,January,1998
